In [13]:
"""
"""
from __future__ import print_function
import os
import argparse
import pickle
from datetime import datetime
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

# tensorflow/keras imports 
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.regularizers import l1
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.losses import CategoricalCrossentropy

# qkeras and pruning imports 
from qkeras.qlayers import QDense, QActivation
from qkeras import QBatchNormalization
from qkeras.quantizers import quantized_bits, quantized_relu
from qkeras.utils import _add_supported_quantized_objects
from tensorflow.keras.models import load_model
from qkeras.utils import _add_supported_quantized_objects

from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks, pruning_schedule
from tensorflow_model_optimization.sparsity.keras import strip_pruning
import tensorflow_model_optimization as tfmot

## Helper functions

In [14]:
def get_checkpoint_filename(checkpoint_dir, experiment_num, checkpoint_filename):
    return os.path.join(checkpoint_dir, f'{experiment_num}', checkpoint_filename)

def get_callbacks(checkpoint_exp_filename, prune):
    callbacks = [
            ModelCheckpoint(
            checkpoint_exp_filename,
            monitor="val_loss",
            verbose=0,
            save_best_only=True,
            save_weights_only=False,
            save_freq="epoch",
        ),
        ReduceLROnPlateau(patience=75, min_delta=1**-6),
    ]
    if prune:
        callbacks.append(pruning_callbacks.UpdatePruningStep())
    return callbacks

def load_checkpoint(filename):
    co = {}
    _add_supported_quantized_objects(co)
    model = load_model(filename, custom_objects=co, compile=False)
    return model

def one_hot_encode(data):
    y_encoded = np.zeros([data.shape[0],2], dtype=np.int32)
    for idx, x in enumerate(data):
        if x == 1:
            y_encoded[idx][1] = 1
        else:
            y_encoded[idx][0] = 1
    return y_encoded

In [15]:
def load_data(data_dir, window_start, window_end, bit_shift):
    print('#########################################################')
    print(f'Loading data from dir: {data_dir}')
    print('\tWindow start:', window_start)
    print('\tWindow end:', window_end)
    print('#########################################################')
    X_train_val = np.load(os.path.join(data_dir, f'X_train_{window_start}_{window_end}.npy'))
    X_test = np.load(os.path.join(data_dir, f'X_test_{window_start}_{window_end}.npy'))    
    y_train_val = np.load(os.path.join(data_dir, f'y_train.npy'))
    y_test = np.load(os.path.join(data_dir, f'y_test.npy'))

    y_train_val = one_hot_encode(y_train_val)
    y_test = one_hot_encode(y_test)

    if bit_shift:
        shift = 3
        print(X_train_val.max())
        X_train_val = np.right_shift(X_train_val.astype(np.int32), shift)
        X_test = np.right_shift(X_test.astype(np.int32), shift)
        print(X_train_val.max())

    return X_train_val, X_test, y_train_val, y_test


#########################################################
# Model
#########################################################
def build_model(model_type, input_shape, quantize, is_pruning):
    model = Sequential()
    # build model 
    if model_type == 'mlp':
        # csr = range(0, 770)
        # sr = len(csr)
        # hn = sr*2 * 1
        sr = int(input_shape/2)
        hn = sr * 2
        if quantize:
            model.add(QDense(int(hn/8), input_shape=(sr*2,), name='fc1', kernel_quantizer=quantized_bits(6,0,alpha=1), bias_quantizer=quantized_bits(6,0,alpha=1),))
            model.add(Activation('relu'))
            model.add(BatchNormalization())
            model.add(QDense(2, name='fc2', kernel_quantizer=quantized_bits(6,0,alpha=1), bias_quantizer=quantized_bits(6,0,alpha=1)))
            model.add(Activation('relu'))
        else:
            model.add(Dense(int(hn/8), activation='relu', input_shape=(sr*2,)))
            model.add(BatchNormalization())
            model.add(Dense(2, activation='relu'))
    elif model_type == 'single':
        if quantize: 
            model.add(QDense(2, input_shape=(input_shape,), name='fc1', kernel_quantizer=quantized_bits(6,0,alpha=1), bias_quantizer=quantized_bits(6,0,alpha=1),))
            model.add(BatchNormalization())
        else:
            model.add(Dense(2, input_shape=(input_shape,), name='fc1'))
            model.add(BatchNormalization())
    # pruning options 
    if is_pruning == True:
        pruning_params = {"pruning_schedule": pruning_schedule.ConstantSparsity(0.85, begin_step=200, frequency=100)}
        # Define model for pruning
        pruning_params = {'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50, final_sparsity=0.80, begin_step=200, end_step=1000)}
        model = prune.prune_low_magnitude(model, **pruning_params)
    return model 


In [16]:
#########################################################
# Training 
#########################################################
def train(model, X_train_val, y_train_val, lr, batch_size, epochs, val_split, callbacks):
    initial_learning_rate = lr
    opt = Adam(learning_rate=initial_learning_rate)
    model.compile(optimizer=opt, loss=CategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
    history = model.fit(X_train_val, 
              y_train_val, 
              batch_size=batch_size,
              epochs=epochs, 
              validation_split=val_split, 
              shuffle=True, 
              callbacks=callbacks
    )
    return history



In [30]:
def log_metrics(model, model_type, window_start, window_end, experiment_number, quantize, prune, train_acc, test_acc):
    # model parameters and sparsity 
    model_params_count = model.count_params()
    zero_params_count = 0
    for layer in model.layers:
        if len(layer.get_weights()) > 0:  # Some layers may not have weights
            params = layer.get_weights()
            for param in params:
                zero_params_count += np.sum(param == 0)

    print(f'Number of parameters: {model_params_count}')
    print(f'Number of zero parameters: {zero_params_count}')

    # save to file 
    with open('notebook_dse.txt', 'a') as file:
        quantize = 1 if quantize == True else 0
        prune = 1 if prune == True else 0
        run_results = f'{model_type}_{window_start}_{window_end}, {experiment_number}, {quantize}, {prune}, {window_start}, {window_end}, {train_acc}, {test_acc}, {model_params_count}, {zero_params_count}\n'
        file.write(run_results)

def evaluate_model(model, checkpoint_exp_filename, history, X_train, y_train, X_test, y_test):
    if prune == True:
        model = strip_pruning(model)

    print(f'Saved checkpoint to: {checkpoint_exp_filename}')
    history_file = checkpoint_exp_filename.replace(".h5", "-history.pkl")
    print(f'Saving history to: {history_file}')
    with open(history_file, 'wb') as file_pi:
        pickle.dump(history.history, file_pi)

    # get test accuracy
    model = load_checkpoint(checkpoint_exp_filename)
    y_pred = model.predict(X_test)
    
    test_acc = accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1))
    print('Keras Test Accuracy: {}'.format(test_acc))

    # get train accuracy
    y_pred = model.predict(X_train)
    train_acc = accuracy_score(np.argmax(y_train, axis=1), np.argmax(y_pred, axis=1))
    print('Keras Train Accuracy: {}'.format(train_acc))

    return train_acc, test_acc
    

# Training parameters

In [31]:
# data options 
data_dir = '../data/malab_05272024/npz'
batch_size = 12800
window_start = 0
window_end = 770
bitshift_data = False
# training options 
max_epochs = 50
val_split = 0.03
lr = 1e-2
# model options 
model_type = 'single'   # options: [mlp, single]
quantize = False
prune = False
# saving options 
checkpoint_dir = '../checkpoints'
checkpoint_filename = 'model_best.h5'
experiment_number = 0
# space exploration parameters 
model_types = ['single', 'mlp']
num_experiments = 3
quantize = True
prune = True
end_windwows = list(range(150, 751, 50))   # [150, 200, 250... 750]
start_windows = [100] * len(end_windwows)  # [100, 100, 100... 100]

In [32]:
for model_type in model_types:
    for start, end in zip(start_windows, end_windwows):
        # load data 
        X_train, X_test, y_train, y_test = load_data(data_dir, start, end, bitshift_data)
        for exp_number in range(num_experiments):
            # build model 
            # model = build_model(model_type, int((end-start)*2), quantize=False, is_pruning=False)
            # model.build((batch_size, int((end-start)*2)))
            model = build_model(model_type, X_train.shape[1], quantize=False, is_pruning=False)
            model.build((batch_size, X_train.shape[1]))

            # train 
            checkpoint_exp_filename = get_checkpoint_filename(checkpoint_dir, experiment_number, checkpoint_filename)
            callbacks = get_callbacks(checkpoint_exp_filename, prune)
            history = train(model, X_train, y_train, lr, batch_size, max_epochs, val_split, callbacks)
            # eval 
            train_acc, test_acc = evaluate_model(model, checkpoint_exp_filename, history, X_train, y_train, X_test, y_test)
            log_metrics(model, model_type, start, end, exp_number, False, False, train_acc, test_acc)

        if quantize:
            for exp_number in range(num_experiments):
                # build model 
                model = build_model(model_type, X_train.shape[1], quantize=True, is_pruning=False)
                model.build((batch_size, X_train.shape[1]))
                # train 
                checkpoint_exp_filename = get_checkpoint_filename(checkpoint_dir, experiment_number, checkpoint_filename)
                callbacks = get_callbacks(checkpoint_exp_filename, prune)
                history = train(model, X_train, y_train, lr, batch_size, max_epochs, val_split, callbacks)
                # eval 
                train_acc, test_acc = evaluate_model(model, checkpoint_exp_filename, history, X_train, y_train, X_test, y_test)
                log_metrics(model, model_type, start, end, exp_number, True, False, train_acc, test_acc)

#########################################################
Loading data from dir: ../data/malab_05272024/npz
	Window start: 100
	Window end: 150
#########################################################
Epoch 1/50
69/69 [==============================] - 1s 7ms/step - loss: 0.5833 - accuracy: 0.7006 - val_loss: 0.5933 - val_accuracy: 0.7155 - lr: 0.0100
Epoch 2/50
69/69 [==============================] - 0s 4ms/step - loss: 0.5559 - accuracy: 0.7221 - val_loss: 0.5675 - val_accuracy: 0.7160 - lr: 0.0100
Epoch 3/50
69/69 [==============================] - 0s 4ms/step - loss: 0.5556 - accuracy: 0.7222 - val_loss: 0.5629 - val_accuracy: 0.7184 - lr: 0.0100
Epoch 4/50
69/69 [==============================] - 0s 4ms/step - loss: 0.5556 - accuracy: 0.7224 - val_loss: 0.5627 - val_accuracy: 0.7176 - lr: 0.0100
Epoch 5/50
69/69 [==============================] - 0s 4ms/step - loss: 0.5556 - accuracy: 0.7225 - val_loss: 0.5614 - val_accuracy: 0.7188 - lr: 0.0100
Epoch 6/50
69/69 [===============